In [42]:
import numpy as np
equipment_dict = {
    0: "Feed",
    1: "Mixer",
    2: "Splitter",
    3: "Heater",
    4: "Cooler",
    5: "HeatExchanger",
    6: "Pump",
    7: "CSTR",
    8: "PFR",
    9: "Flash",
    10: "DistillationColumn",
    11: "DistillationColumnwithRecycle",
    12: "Compressor",
    13: "Turbine",
    14: "Product",
    15: "Subbranch 1 starter",
    16: "Subbranch 2 starter",
    17: "Subbranch 3 starter",
    18: "Subbranch 1",
    19: "Subbranch 2",
    20: "Subbranch 3",
    21: "HeatExchanger A",
    22: "HeatExchanger B",
    23: "Minorfeed",
    24: "Minorproduct",
    25: "End",
}
string_integer_dict = {
    0: "F",
    1: "M",
    2: "S",
    3: "H",
    4: "C",
    5: "Hx",
    6: "Pump",
    7: "Cstr",
    8: "Pfr",
    9: "Flash",
    10: "DC",
    11: "DCr",
    12: "Comp",
    13: "T",
    14: "P",
    15: "-1",
    16: "-2",
    17: "-3",
    18: "1",
    19: "2",
    20: "3",
    21: "Hxa",
    22: "Hxb",
    23: "f",
    24: "p",
    25: "",
}

equipment1 = [0, 18, 1, 3, 7, 7, 7, 7, 7, 7, 7, 7, 4, 11, 14, 15, 6, 18, 25] # CSTR design
equipment2 = [0,18,1,3,8,4,11,14,15,6,18,25] # PFR design
equipment3 = np.array([0,13,21,19,1,22,4,12,2,3,18,1,2,3,20,1,3,14,15,22,18,16,21,2,13,19,17,20,25]) # Nabil ED5 Ta1bAC-2H2b2-3H3a-1T3H


def equipment_to_string(equipment):
    equipment_string = []
    for eq in equipment:
        equipment_string.append(string_integer_dict[eq])
    return ''.join(equipment_string)

equipment_strings = [equipment_to_string(equipment1), equipment_to_string(equipment2), equipment_to_string(equipment3)]
print("Equipment strings:")
for eq_str in equipment_strings:
    print(eq_str)

#string to equipment mapping requires dividing the string into chunks between capital letters also 1 and -1 characters are critical to identify subbranches




def connectivity(equipment,equipment_dict,max_length=30):
    """
    This function takes a list of equipment and checks the connectivity rules are satisfied.
    Rules:
    1. The first equipment must be a feed.
    2. The last equipment must be an end.
    3. The product equipment must be present.
    4. The length of the equipment list must not exceed max_length.
    5. For every subbranch starter, there must be 2 subbranch tokens to indicate the end of the subbranch and connection point.
    6. For every subbranch starter, it should find the corresponding subbranch end before any other subbranch starter.
    7. The same subbranch token can not be followed by the same subbranch starter token.
    8. The subbranch tokens must be followed by a mixer if the same subbranch starter tokens have not been used before.
    9. There should be same number of subbranch starter tokens and multiple input/output equipment tokens.
    """
    if len(equipment) > max_length:
        raise ValueError(f"Equipment list exceeds maximum length of {max_length}.")
    
    if equipment[0] != 0:
        raise ValueError("The first equipment must be a feed.")
    
    if equipment[-1] != len(equipment_dict) - 1:
        raise ValueError("The last equipment must be an end.")

    if 14 not in equipment:
        raise ValueError("The product equipment must be present.")
    
    subbranch_starters_and_positions = [(i,e) for i, e in enumerate(equipment) if e in [15, 16, 17]]
    subbranch_ends_and_positions = [(i,e) for i, e in enumerate(equipment) if e in [18, 19, 20]]
    
    if 2*len(subbranch_starters_and_positions) != len(subbranch_ends_and_positions):
        print(len(subbranch_starters_and_positions), len(subbranch_ends_and_positions))
        raise ValueError("For every subbranch starter, there must be 2 subbranch tokens to indicate the end of the subbranch and connection point.")
    
connectivity(equipment1, equipment_dict)
connectivity(equipment2, equipment_dict)
connectivity(equipment3, equipment_dict)



Equipment strings:
F1MHCstrCstrCstrCstrCstrCstrCstrCstrCDCrP-1Pump1
F1MHPfrCDCrP-1Pump1
FTHxa2MHxbCCompSH1MSH3MHP-1Hxb1-2HxaST2-33
